In [1]:
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.models import Model
from keras.layers import Flatten
import os

Using TensorFlow backend.


In [2]:
base_resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224, 224, 3), classes=1000)
flatFeaturesLayer = Flatten()(base_resnet.output)
featureExtractor = Model(inputs=base_resnet.input, outputs=flatFeaturesLayer)
featureExtractor.compile('sgd', 'mse')

define dataset path

In [3]:
datasetPath = '../datasets/coco/train2017/'

create list of paths for every image in dataset

In [4]:
imgpaths = [os.path.join(root, f) 
            for root, _, files in os.walk(datasetPath) 
            for f in files if f.endswith('.jpg')]
print(len(imgpaths))
imgpaths = imgpaths[0: 1000]

In [5]:
with open('imgpaths.txt', 'w') as f:
    for b in imgpaths:
        f.write(b + '\n')
vectorsPath = 'data/vectors_resnet/'        

define function to extract vectors for every image in dataset in batches of 64 images

In [6]:
def extractAndSaveVectorsForBatch(batchImgPaths):
    batchSize = len(batchImgPaths)
    batch = np.zeros((batchSize, 224, 224, 3), dtype=np.float)
    
    for i in range(0, batchSize):
        img = image.img_to_array(image.load_img(batchImgPaths[i], target_size=(224, 224)))
        batch[i] = preprocess_input(img)
        
    batchVectors = featureExtractor.predict_on_batch(batch).astype(np.float16)
    for i in range(0, batchSize):
        np.save(batchImgPaths[i].replace(datasetPath, vectorsPath) + '.npy', batchVectors[i])

extract and save in .npy format vector for every emage in dataset

In [8]:
batchSize = 64
printEvery = 8
imgCount = len(imgpaths)
for i in range(0, imgCount, batchSize):
    extractAndSaveVectorsForBatch(imgpaths[i: i+batchSize])
    if not i % printEvery*batchSize:
        print('extracted vectors for %s images of %s' % (min(i+batchSize, imgCount), imgCount))
print('extracted vectors for every image')

extracted vectors for every image


load every vector in one matrix 

In [10]:
imgVectors = []
for imgpath in imgpaths:
    vectorpath = imgpath.replace(datasetPath, vectorsPath) + '.npy'
    imgVectors.append(np.load(vectorpath))
imgVectors = np.array(imgVectors)
print('loaded all image vectors')

loaded all image vectors


define function that returns cosine distance for every vector in dataset

In [11]:
import scipy


def cos_cdist(matrix, vector):
    v = vector.reshape(1, -1)
    return scipy.spatial.distance.cdist(matrix, v, 'cosine').reshape(-1)

define function that loads image using given imgpath, extract feature vector for it and find paths of images that have nearest features vectors using cosine distabce metric

In [12]:
imgpaths = np.array(imgpaths)


def findNearestImages(imgPath, topn=4):
    img = preprocess_input(image.img_to_array(image.load_img(imgPath, target_size=(224, 224))))
    img = np.expand_dims(img, axis=0)
    searchImgVector = featureExtractor.predict(img)[0]
    imgDistances = cos_cdist(imgVectors, searchImgVector)
    nearestIds = np.argsort(imgDistances)[:topn].tolist()
    print(nearestIds)
    nearestImgPaths = imgpaths[nearestIds]
    print(nearestImgPaths)
    return nearestImgPaths

In [15]:
searchImgPath = 'imgs/sport1.jpg'
similarImagesPaths = findNearestImages(searchImgPath, topn=4)
similarimages = []
# print(similarImagesPaths)
for imgPath in similarImagesPaths:
    similarimages.append(image.load_img(imgPath, target_size=(224, 224)))

IndexError: too many indices for array

In [14]:
similarimages[1]

NameError: name 'similarimages' is not defined